In [23]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder , StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score,accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder , StandardScaler , MinMaxScaler , RobustScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score,accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from collections import Counter
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, precision_score, classification_report
from sklearn.metrics import recall_score




### Read Data

In [24]:

diabetes = pd.read_csv("diabetes_binary_health_indicators_BRFSS2015.csv")
diabetes.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


### Remove Outliers

In [25]:
def drop_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    df_cleaned = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df_cleaned

diabetes_cleaned = drop_outliers_iqr(diabetes, 'BMI')
diabetes_cleaned = drop_outliers_iqr(diabetes_cleaned, 'GenHlth')

### Split Data

In [26]:

x_train, x_test = train_test_split(diabetes_cleaned, test_size=0.20, random_state=42)
x_test,y_test = x_test.drop(['Diabetes_binary'],axis=1),x_test['Diabetes_binary']
x_train,y_train = x_train.drop(['Diabetes_binary'],axis=1),x_train['Diabetes_binary']


### Search For Best Parameters Wit Cross-Validation with Smothing

In [ ]:
# smote = SMOTE(random_state=42)
# x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

# recall_class1_scorer = make_scorer(
#     recall_score, 
#     pos_label=1,
#     average='binary'
# )
# randomforest =RandomForestClassifier()

# rf_param_grid = [
#     {
#         'n_estimators': [100, 200],
#         'max_depth': [10, 20, 30],
#         'min_samples_split': [2, 5, 10],
#         'min_samples_leaf': [2, 4],
#         'class_weight': [None, 'balanced'],
#     },
#     {
#         'n_estimators': [100, 200],
#         'max_features': ['sqrt', 'log2'],
#         'bootstrap': [True, False],
#         'class_weight': [None, 'balanced'],
#     }
# ] 

# rf_grid_search = GridSearchCV(randomforest, rf_param_grid, cv=5, scoring=recall_class1_scorer, n_jobs=-1, verbose=2)
# rf_grid_search.fit(x_train_resampled, y_train_resampled)


# y_pred = rf_grid_search.predict(x_test)

# print("Best Parameters:", rf_grid_search.best_params_)
# print("Best estimator:", rf_grid_search.best_estimator_)
# print("Best score:", rf_grid_search.best_score_)
# print("\nTest Accuracy: {:.2%}".format(accuracy_score(y_test, y_pred)))


Fitting 5 folds for each of 176 candidates, totalling 880 fits
Best Parameters: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Best estimator: RandomForestClassifier(class_weight='balanced', max_depth=10,
                       min_samples_leaf=2, n_estimators=200)
Best score: 0.873347769068895

Test Accuracy: 82.06%


### TEST 1 USE Smothing

In [ ]:
smote = SMOTE(random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

model = RandomForestClassifier(class_weight='balanced', max_depth=10,
                       min_samples_leaf=2, n_estimators=200)

model.fit(x_train_resampled, y_train_resampled)

y_train_pred = model.predict(x_train_resampled)
y_test_pred = model.predict(x_test)

train_accuracy = accuracy_score(y_train_resampled, y_train_pred) * 100
test_accuracy = accuracy_score(y_test, y_test_pred) * 100

print(f"Training Accuracy: {train_accuracy:.2f}%")
print(f"Test Accuracy: {test_accuracy:.2f}%")

print(classification_report(y_test, y_test_pred, target_names=['No Diabetes', 'Diabetes']))

def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['No Diabetes', 'Diabetes'],
                yticklabels=['No Diabetes', 'Diabetes'])
    plt.title(title)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

# plot_confusion_matrix(y_train_resampled, y_train_pred, "Training Set Confusion Matrix")
# plot_confusion_matrix(y_test, y_test_pred, "Test Set Confusion Matrix")


Training Accuracy: 87.55%
Test Accuracy: 82.05%
              precision    recall  f1-score   support

 No Diabetes       0.93      0.86      0.89     40903
    Diabetes       0.34      0.52      0.41      5689

    accuracy                           0.82     46592
   macro avg       0.64      0.69      0.65     46592
weighted avg       0.86      0.82      0.84     46592



### Search For Best Parameters Wit Cross-Validation 

In [ ]:
# recall_class1_scorer = make_scorer(
#     recall_score, 
#     pos_label=1,
#     average='binary'
# )
# randomforest =RandomForestClassifier()

# rf_param_grid = [
#     {
#         'n_estimators': [100, 200],
#         'max_depth': [10, 20, 30],
#         'min_samples_split': [2, 5, 10],
#         'min_samples_leaf': [2, 4],
#         'class_weight': [None, 'balanced'],
#         'criterion': ['gini', 'entropy']
#     },
#     {
#         'n_estimators': [100, 200],
#         'max_features': ['sqrt', 'log2'],
#         'bootstrap': [True, False],
#         'class_weight': [None, 'balanced'],
#         'criterion': ['gini', 'entropy']
#     }
# ] 

# rf_grid_search = GridSearchCV(randomforest, rf_param_grid, cv=5, scoring=recall_class1_scorer, n_jobs=-1, verbose=2)
# rf_grid_search.fit(x_train, y_train)

# y_pred = rf_grid_search.predict(x_test)

# print("Best Parameters:", rf_grid_search.best_params_)
# print("Best estimator:", rf_grid_search.best_estimator_)
# print("Best score:", rf_grid_search.best_score_)
# print("\nTest Accuracy: {:.2%}".format(accuracy_score(y_test, y_pred)))

### TEST 2 NOT USE Smothing

In [29]:

model_rf = RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=10, min_samples_leaf=4, min_samples_split=10,
                       n_estimators=100, random_state=42)
model_rf.fit(x_train, y_train)

y_train_pred = model_rf.predict(x_train)
y_test_pred = model_rf.predict(x_test)

train_accuracy = accuracy_score(y_train, y_train_pred) * 100
test_accuracy = accuracy_score(y_test, y_test_pred) * 100

print(f"Training Accuracy: {train_accuracy:.2f}%")
print(f"Test Accuracy: {test_accuracy:.2f}%")

def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['No Diabetes', 'Diabetes'],
                yticklabels=['No Diabetes', 'Diabetes'])
    plt.title(title)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

# plot_confusion_matrix(y_train, y_train_pred, "Training Set Confusion Matrix")
# plot_confusion_matrix(y_test, y_test_pred, "Test Set Confusion Matrix")

print("\nClassification Report (Test Set):")
print(classification_report(y_test, y_test_pred,target_names=['No Diabetes','Diabetes']))

Training Accuracy: 72.91%
Test Accuracy: 72.24%

Classification Report (Test Set):
              precision    recall  f1-score   support

 No Diabetes       0.96      0.71      0.82     40903
    Diabetes       0.28      0.78      0.41      5689

    accuracy                           0.72     46592
   macro avg       0.62      0.75      0.61     46592
weighted avg       0.88      0.72      0.77     46592



### Test 3 Change Threshold

In [34]:
y_train_probs = model_rf.predict_proba(x_train)[:, 1]
y_probs = model_rf.predict_proba(x_test)[:, 1]

threshold = 0.4
y_train_pred = (y_train_probs >= threshold).astype(int)
y_test_pred = (y_probs >= threshold).astype(int)

train_accuracy = accuracy_score(y_train, y_train_pred) * 100
test_accuracy = accuracy_score(y_test, y_test_pred) * 100

print(f"Training Accuracy with Threshold {threshold}: {train_accuracy:.2f}%")
print(f"Test Accuracy with Threshold {threshold}: {test_accuracy:.2f}%")

print("\nClassification Report (Test Set) with Threshold =", threshold)
print(classification_report(y_test, y_test_pred, target_names=['No Diabetes', 'Diabetes']))

Training Accuracy with Threshold 0.4: 63.83%
Test Accuracy with Threshold 0.4: 63.35%

Classification Report (Test Set) with Threshold = 0.4
              precision    recall  f1-score   support

 No Diabetes       0.97      0.60      0.74     40903
    Diabetes       0.23      0.88      0.37      5689

    accuracy                           0.63     46592
   macro avg       0.60      0.74      0.56     46592
weighted avg       0.88      0.63      0.70     46592



### Test 4 Get the top 10 Features

In [35]:

selector = SelectKBest(f_classif, k=10)
X_new = selector.fit_transform(x_train, y_train)

selected_features = x_train.columns[selector.get_support()]
print("Best features:", selected_features)

Best features: Index(['HighBP', 'HighChol', 'BMI', 'HeartDiseaseorAttack', 'GenHlth',
       'PhysHlth', 'DiffWalk', 'Age', 'Education', 'Income'],
      dtype='object')


In [36]:
X_train_selected = selector.transform(x_train)
X_test_selected = selector.transform(x_test)
model_selected_features = RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=10, min_samples_leaf=4, min_samples_split=10)
model_selected_features.fit(X_train_selected, y_train)

y_train_pred = model_selected_features.predict(X_train_selected)
y_test_pred = model_selected_features.predict(X_test_selected)

train_accuracy = accuracy_score(y_train, y_train_pred) * 100
test_accuracy = accuracy_score(y_test, y_test_pred) * 100

print(f"Training Accuracy: {train_accuracy:.2f}%")
print(f"Test Accuracy: {test_accuracy:.2f}%")
print(classification_report(y_test, y_test_pred, target_names=['No Diabetes', 'Diabetes']))


Training Accuracy: 72.77%
Test Accuracy: 72.12%
              precision    recall  f1-score   support

 No Diabetes       0.96      0.71      0.82     40903
    Diabetes       0.27      0.78      0.40      5689

    accuracy                           0.72     46592
   macro avg       0.62      0.75      0.61     46592
weighted avg       0.87      0.72      0.77     46592

